# Tasks

* Implement vanilla autoencoder
* Train it on MNIST dataset MNIST
* Display digits recovered dy AE
* Display distribution of embeddings in latent space

#Dataset

http://yann.lecun.com/exdb/mnist/

The MNIST database of handwritten digits, has a training set of 60,000 examples, and a test set of 10,000 examples.

The images were centered in a 28x28 image by computing the center of mass of the pixels.



In [ ]:
import torch, torchvision
from torchvision.datasets import MNIST
from torchvision import transforms, utils


transf = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize( (0.1307) , (0.3081) )
                             ])

train_dataset = MNIST('mnist',
                      train = True,
                      download = True,
                      transform = transf
                      )

test_dataset = MNIST('mnist',
                     train = False,
                     download = True,
                     transform = transf
                     )

Display some samples along with corresponding labels

In [ ]:
import matplotlib.pyplot as plt
from torch.utils.data import  DataLoader

# Helper method
def show(grid):
  plt.axis("off")
  plt.imshow(grid.permute(1,2,0).numpy())
  plt.show()

vis_dataloader = DataLoader(test_dataset, batch_size = 8, shuffle=True)
dataiter = iter(vis_dataloader)
example_batch = next(dataiter) # img1,  label

# display the data
grid = torchvision.utils.make_grid(example_batch[0])
show(grid)
print(example_batch[1].numpy())

# Model

Implement vanilla autoencoder model.

In [ ]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, latent_size):
      super().__init__()
      self.latent_size = latent_size

      # Put your code here

    def forward(self,x):

      # Put your code here

      return  embedding


class Decoder(nn.Module):
    def __init__(self, latent_size):
      super().__init__()
      self.latent_size = latent_size

      # Put your code here


    def forward(self,x):

      # Put your code here

      return recovered

##Smoke test

In [ ]:
encoder = Encoder(2)
dummy = torch.randn((1,1,28,28))
print("Encoder In",dummy.shape)
embedding = encoder(dummy)
print("encoder Out",embedding.shape)


decoder =Decoder(2)
recovered = decoder(embedding)
print("Decoder out",recovered.shape)

assert( dummy.shape == recovered.shape) ,"Decoder out shape must be equal to input shape"

###AE

Implement AutoEncoder class.
Use Decoder and Encoder classes implemented early.

In [ ]:
class AutoEncoder(nn.Module):
  def __init__(self, latent_size):
    super().__init__()

    # Put your code here

  def forward(self,x):
    # Put your code here

    return recovered_x, embedding

# Train

Define dataloaders

In [ ]:
train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=128)
test_dataloader = DataLoader(test_dataset,shuffle=False,batch_size=256)

Define main training routine and train your model

In [ ]:
from torch import optim
from torch.utils.tensorboard import SummaryWriter

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Put your code here

## Let's test the model

Compare original and recovered digits.

In [ ]:
# put your code here

Show embeddings positions in latent space

In [ ]:
# put your code here

# Conclusion

...

# Ideas for extra work

* Find the best latent space size
* Implement noise filtration with AE
* Test vector arithmetic in laent space
* Implemet VAE
** Use Autoencoder class as base class
** Implement VAE Loss class
** Plot embeddings manifold in VAE latent space
** Compare decoding results VAE latent space with vanilla Autoencoder results
* Replace reconstruction loss from MSE to BCE
* Implement Conditional Autoencoder or CVAE